In [1]:
from diff2vec.numba.diffuser import create_diffusion_graph_numba
from test_utils.samples import get_random_graph_matrix
import numpy as np
import networkx as nx
import time

This noetebook containsts time performance tests for different difussion graph creating implementations.

In [2]:
def create_diffusion_graph(G, start_node, number_of_nodes, seed=2137):
    '''just for the time tests: version of diff2vec.diffuser.create_diffusion_graph, but returning vertices and edges instead of nx.Graph'''
    np.random.seed(seed)
    G_diff_vertices = [start_node]
    G_diff_edges = []

    for _ in range(number_of_nodes-1):
        
        while True:
            u = np.random.choice(G_diff_vertices)
            neighs = list(G.neighbors(u))
            not_taken_neighs = np.setdiff1d(neighs, G_diff_vertices)
            if len(not_taken_neighs)>0:
                break
    
        w = np.random.choice(not_taken_neighs)

        G_diff_vertices.append(w)
        G_diff_edges.append((u, w))

    return G_diff_vertices, np.array(G_diff_edges)

In [4]:
def diffuser_unit_test(N):
    print(f"N: {N}")
    A = get_random_graph_matrix(N)
    G = nx.from_numpy_array(A)
    print("Generation finished")
    
    start = time.time()
    base = create_diffusion_graph(G, 0, int(0.3*N))
    end = time.time()
    print(f"Time base: {end-start}")

    start = time.time()
    numba =  create_diffusion_graph_numba(A, 0, int(0.3*N))
    end = time.time()
    print(f"Time numba: {end-start}")

    assert (base[0] == numba[0]).all()
    assert (base[1] == numba[1]).all()

In [6]:
diffuser_unit_test(1000)

N: 1000
Generation finished
Time base: 0.03700399398803711
Time numba: 0.0019986629486083984


In [464]:
diffuser_unit_test(3000)

N: 3000
Generation finished
Time base: 0.32610321044921875
Time numba: 0.013834238052368164


In [466]:
diffuser_unit_test(5000)

N: 5000
Generation finished
Time base: 1.1157045364379883
Time numba: 0.0485689640045166


In [467]:
diffuser_unit_test(10000)

N: 10000
Generation finished
Time base: 5.5806097984313965
Time numba: 0.2123267650604248
